<a href="https://colab.research.google.com/github/mloyorev/Theory/blob/main/9_OptimalSavingsJax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install quantecon
import quantecon as qe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 1.1 MB/s eta 0:00:00


In [5]:
import numpy as np
import quantecon as qe
import jax
import jax.numpy as jnp
from collections import namedtuple
import matplotlib.pyplot as plt

One difference between `numpy` and `jax` is that, when running on a GPU, `jax` uses 32 bit floats by default.  This is standard for GPU computing and can lead to significant speed gains with small loss of precision.

However, **for some calculations precision matters**. In this case we will use 64 bit floats with `jax`  in order to match with `7_OptimalSavingsNumba.ipynb` (this is because `numpy` use double precision floating point numbers, also known as `float64`).

In [3]:
jax.config.update("jax_enable_x64", True)

As in the previuous notebooks, we will define the following functions:


*   `supremum_norm`
*   `succesive_approx`



In [6]:
def supremum_norm(x,x_new):
  return np.max(jnp.abs(x_new - x))

def successive_approx(T,
                      x_0,
                      M,
                      tolerance=1e-6,
                      max_iter=10_000,
                      print_step=25,
                      verbose=False):

    x = x_0
    error = tolerance + 1
    k = 1

    while error > tolerance and k <= max_iter:
        x_new = T(x)
        error = M(x, x_new)
        if verbose and k % print_step == 0:
            print(f"Completed iteration {k} with error {error}.")
        x = x_new
        k += 1

    if error > tolerance:
        print(f"Warning: Iteration hit upper bound {max_iter}.")

    elif verbose:
        print(f"Terminated successfully in {k} iterations.")
    return x

# **Optimal savings model with Google Jax**

As we have alredy mentioned, the **Bellman equation** of this model is given by

$$V(w,y)=\max_{w'}u(Rw+y-w')+\beta\sum_{y'\in Y}V(w',y')Q(y',y)$$

where

$$u(c)=\frac{c^{1+\gamma}}{1+\gamma}$$

**Details on the model assumptions and detailed derivation of the Bellman equation** comes in the `7_OptimalSavingsNumba` notebook.

The main objetive of this notebook is to prove **how `jax` improve the performance** of the model solvers that we create in `7_OptimalSavingsNumba`

In [8]:
Model = namedtuple('Model',('beta', 'R', 'gamma', 'w_grid', 'y_grid', 'Q'))

def create_consumption_model(R=1.01,                    # Gross interest rate
                             beta=0.98,                 # Discount factor
                             gamma=2.5,                 # CRRA parameter
                             w_min=0.01,                # Min wealth
                             w_max=5.0,                 # Max wealth
                             w_size=150,                # Grid side
                             rho=0.9,                   # Persistance of the shock
                             nu=0.1,                    # Volatility of stochastic error (sigma)
                             y_size=100):

    w_grid = np.linspace(w_min, w_max, w_size)
    mc = qe.tauchen(rho= rho, sigma = nu, n = y_size)
    y_grid, Q = np.exp(mc.state_values), mc.P
    return Model(beta=beta, R=R, gamma=gamma, w_grid=w_grid, y_grid=y_grid, Q=Q)

In order to use `jax` **we have to create a `jax`-compatible version** of the consumption model.

This `jax`-compatible version of the consumption model:


*  Break up parameters into static and nonstatic components
*   Move the arrays to a device, for example the GPU or the TPU.

Moving the arrays to a device is very **important because:**


1.   GPUs (Graphics Processing Unit) and TPUs (Tensor Processing Unit) are designed to **perform parallel calculations** much faster than CPUs (Central Processing Unit).
2.   By moving data to the device, JAX is allowed to perform **parallel calculations** on specialized hardware, which can significantly speed up calculation performance and efficiency.





In [9]:
# Build a JAX-compatible version of the consumption model
def create_consumption_model_jax():

    model = create_consumption_model()          # Create an instance of the model
    beta, R, gamma, w_grid, y_grid, Q = model   # Unpacked model parameters

    # Break up parameters into static and nonstatic components
    constants = beta, R, gamma
    sizes = len(w_grid), len(y_grid)
    arrays = w_grid, y_grid, Q

    arrays = tuple(map(jax.device_put, arrays)) # Shift arrays to the device (e.g., GPU and TPU)

    return constants, sizes, arrays

Now we compute the RHS of the **Bellman equation**.

In [10]:
def B(v, constants, sizes, arrays):

    # Unpack model parameters
    β, R, γ = constants
    w_size, y_size = sizes
    w_grid, y_grid, Q = arrays

    # Notice that index:
    #   - i is for current wealth
    #   - j is for current income
    #   - ip is for next period's wealth
    #   - jp is for next period's income

    # Compute current rewards r(w, y, wp) as an array r[i, j, ip] of three dimesions
    w  = jnp.reshape(w_grid, (w_size, 1, 1))    # w[i]   ->  w[i, j, ip]
    y  = jnp.reshape(y_grid, (1, y_size, 1))    # z[j]   ->  z[i, j, ip]
    wp = jnp.reshape(w_grid, (1, 1, w_size))    # wp[ip] -> wp[i, j, ip]
    c = R * w + y - wp                          # Current reward

    # Calculate continuation rewards at all combinations of (w, y, wp)
    v = jnp.reshape(v, (1, 1, w_size, y_size))  # v[ip, jp] -> v[i, j, ip, jp]
    Q = jnp.reshape(Q, (1, y_size, 1, y_size))  # Q[j, jp]  -> Q[i, j, ip, jp]
    EV = jnp.sum(v * Q, axis=3)                 # Sum over last index jp for the expected value

    # Comoute the RHS of the Bellman equation if and only if c>0, otherwise set -np.inf
    return jnp.where(c > 0, c**(1-γ)/(1-γ) + β * EV, -np.inf)